In [ ]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
data = pd.read_csv('baseline_training_data.csv')
data[:5]

In [ ]:
labels = sorted(list(set(data['label_x'])))
labels

In [ ]:
fold = int(0.1*data.shape[0])

for num in range(10):
    w_org = np.array([np.zeros(data.shape[1]-2) for label in labels])
    w_avg = w_org.copy()
    
    if num == 0:
        train_data = data[fold:data.shape[0]]
    elif num == 9:
        train_data = data[:fold*9]
    else:
        train_data = pd.DataFrame(np.concatenate((data[:fold*num], data[fold*(num+1):]), axis=0))
            
    N_times = 5
    T_len = train_data.shape[0]
    step = N_times * T_len

    for t in range(N_times):
        y_true = []
        y_pred = []
        for train in train_data.index:

            row_data = train_data[train_data.index.isin([train])].values[0]

            w_list = [row_data[2:].dot(w.T) for w in w_org]

            pred_label = w_list.index(max(w_list))
            true_label = labels.index(row_data[1])

            if pred_label != true_label:

                w_org[true_label] = w_org[true_label] + row_data[2:]
                w_org[pred_label] = w_org[pred_label] - row_data[2:]

                w_avg[true_label] = w_avg[true_label] + (step/(N_times * T_len)) * row_data[2:]
                w_avg[pred_label] = w_avg[pred_label] - (step/(N_times * T_len)) * row_data[2:]

            step = step -1

    if num == 9:
        pred_data=data[(fold*num):]
    else:
        pred_data=data[(fold*num):(fold*(num+1))]
    
    y_true = []
    y_pred = []
    for pred in pred_data.index:

        row_data = pred_data[pred_data.index.isin([pred])].values[0]

        w_list = [row_data[2:].dot(w.T) for w in w_avg]

        pred_label = w_list.index(max(w_list))

        true_label = labels.index(row_data[1])

        y_true.append(labels[true_label])
        y_pred.append(labels[pred_label])
    clas_report = metrics.classification_report(y_true, y_pred)
    print("Classification Report(Set %s):\n" % (num+1) + clas_report)
    print("Micro-average: ", precision_recall_fscore_support(y_true, y_pred, average='micro'))
    print("Macro-average: ", precision_recall_fscore_support(y_true, y_pred, average='macro'))
    print("Accurancy: %s" % accuracy_score(y_true, y_pred)+'\n')

    with open('results/10_fold_report_unigram_n.csv', 'a+') as csv_file:
        csv_file.write("Classification Report(Set %s):\n" % (num+1)+clas_report+'\n')
        csv_file.write("Micro-average: "+str(precision_recall_fscore_support(y_true, y_pred, average='micro'))+'\n')
        csv_file.write("Macro-average: "+str(precision_recall_fscore_support(y_true, y_pred, average='macro'))+'\n')
        csv_file.write("Accurancy: "+str(accuracy_score(y_true, y_pred))+'\n\n')